In [16]:
# !pip install bs4
# !pip install requests
# !pip install tqdm
# !pip install pandas
# !pip install datetime 
# !pip install json 
# !pip install pymysql
# !pip install sqlalchemy

In [ ]:
import winsound
import numpy as np
# import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib

from webscraper.scrap_modules import scraper_ds as mybib


%load_ext autoreload
%autoreload 2

In [ ]:
# pygame.mixer.init()
# pygame.mixer.music.load("path/to/your/audiofile.mp3")
# pygame.mixer.music.play()
# while pygame.mixer.music.get_busy():
#         pygame.time.Clock().tick(10)  # Wait for the music to finish

In [ ]:
# try:
#     # Your code here
#     x = 1 / 0  # This will cause a ZeroDivisionError
# except Exception as e:
#     print(f"An error occurred: {e}")
#     winsound.PlaySound("alarm2.wav", winsound.SND_FILENAME)

### Import keywords

In [ ]:
keyword_list = mybib.import_keyword_list()
keyword_list

### Scrap Prep
- connect with MySQL to get id's
- create empty DataFrame for saving scraping results
- print results for check

In [ ]:
try:
    with open("webscraper/webscrap_data/soup_dict.pkl", "rb") as file:
        soup_dict = pickle.load(file)
    with open("webscraper/temp_data/soup_dict_backup.pkl", "wb") as file:
        pickle.dump(soup_dict, file)
except Exception as e:
    raise ValueError(f"Error occurred: {e} - soup_dict")      
        
try:
    with open("webscraper/webscrap_data/keyword_dict.pkl", "rb") as file:
        keyword_dict = pickle.load(file)
    with open("webscraper/temp_data/keyword_dict_backup.pkl", "wb") as file:
        pickle.dump(soup_dict, file) 
except Exception as e:
    raise ValueError(f"Error occurred: {e} - keyword_dict")   

    
# scraper_df = mybib.initialize_empty_df()

In [ ]:
# try:
#     with open("webscraper/webscrap_data/database.pkl", "rb") as file:
#         database = pickle.load(file)
#     with open("webscraper/temp_data/database_backup.pkl", "wb") as file:
#         pickle.dump(database, file)
        
#     id_control = database['id'].tolist()
# except:
#     print("No dataframe found.")
#     columns = ['id','title', 'company', 'city', 'posting_date', 'job_description', 'seniority_level', 'job_function', 'industries', 'scraping_date', 'url', 'keyword']
#     database = pd.DataFrame(columns=columns)
#     id_control = []   

In [ ]:

print(len(soup_dict))
print(len(keyword_dict))


In [ ]:
print(soup_dict)
id_control = []
for item in soup_dict.keys():
    id_control.append(item)

In [ ]:
# print("Overview SQL Database")
# display(database.head(10))
# print(database.shape)

# print("Empty Dataframe")
# display(scraper_df.head())

# new_dataframe.shape

In [ ]:
# id_control = database['id'].tolist()
id_control

In [ ]:
# sql_data = mybib.connect_sql_database()
# scraper_df = mybib.initialize_empty_df()

# print("Overview SQL Database")
# display(sql_data.head(5))
# display(sql_data.tail(5))

# print("Empty Dataframe")
# display(scraper_df)

# print("Overview ID control")
# id_control = sql_data['id'].tolist()
# id_control



    

In [ ]:
# new_scraper_df, new_soup_dict, new_keyword_dict = mybib.total_loop_rough_keywords(keyword_list, scraper_df, id_control, soup_dict, keyword_dict)

In [34]:
# berlin all time https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=&distance=25&f_E=2%2C3%2C4&f_JT=F&position=1&pageNum=0
# berlin past month https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=r604800&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0
# germany last week https://www.linkedin.com/jobs/search?keywords={key}&location=Germany&locationId=&geoId=101282230&f_TPR=r604800&f_E=2%2C3%2C4&position=1&pageNum=0
# last month total germany data https://www.linkedin.com/jobs/search?keywords=Data&location=Germany&locationId=&geoId=101282230&f_TPR=r2592000&f_E=2%2C3%2C4&position=1&pageNum=0

In [35]:
len(id_list)


984

In [37]:
# def total_loop_rough_keywords(keyword_list, scraper_df, id_control, soup_dict, keyword_dict): 

# links = {}
try:
    for key in keyword_list:
        first_url = f"https://www.linkedin.com/jobs/search?keywords={key}&location=Germany&locationId=&geoId=101282230&f_TPR=r2592000&f_E=2%2C3%2C4&position=1&pageNum=0"
        key_name = key.replace('%20', " ")
            
        response = requests.get(first_url) # first request for keyword
        response.status_code # 200 status code means OK!
        soup = BeautifulSoup(response.content, "html.parser")
        
        number_of_results = soup.find('span', class_="results-context-header__job-count").text # check number of searching results
        numb = int(number_of_results.replace(",", "").replace("+", ""))

        backend_call_url_list = []
        backend_call_url_list = mybib.create_backend_links(first_url, numb, key_name) # create list with sublinks to select different pages 
        
        
        # with open(f'{key_name}_backend_urls.pkl', 'wb') as file:
        #     pickle.dump(backend_call_url_list, file)
        
        id_array = mybib.get_id_list(backend_call_url_list) # get job id's from all pages
        

        with tqdm(total=len(id_array), desc="Starting") as pbar:
        
            for id in id_array:
                counter = 0
                backupsave = [25, 50, 75, 100, 125, 150, 175, 200, 300] 
                if counter in backupsave:
                    new_dataframe = mybib.export_data(soup_dict, keyword_dict)
                    scraper_df = pd.DataFrame(columns=scraper_df.columns)
                    print("Updated dataframe")
                dynamic_text = f"id {id}" # text for tqdm progress bar status
                pbar.set_description(dynamic_text) # change text
                pbar.update(1)
                
                if str(id) not in id_control:
                    try:
                        
                        # new_row_dict, id_control, response = mybib.get_all_job_information(id, id_control) # 
                        response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}')
                        wait_time = randint(1,500)
                        sleep(wait_time/1000)
                        
                        dict2 = {}
                        dict2["scrap_date"] = date.today()
                        dict2["response"] = response
                        soup_dict[id] = dict2
                        # If the key 'id' doesn't exist, initialize it with a list containing the new key

                        if id in keyword_dict:
                            keyword_dict[id].append(key_name)
                        # If the key 'id' doesn't exist, initialize it with a list containing the new key
                        else:
                            keyword_dict[id] = [key_name]

                    except Exception as e:
                        raise ValueError(f"Error occurred: {e} getting data from id {id}")
                        
                else:
                    pbar.set_description(f"Will skip {id} because is already in the dataset.")
                counter += 1    
        new_dataframe = mybib.export_data(soup_dict, keyword_dict)
        print("\n \n")
except Exception as e:
    print(f"An error occurred: {e}")
    winsound.PlaySound("alarm2.wav", winsound.SND_FILENAME)
    raise ValueError(f"Error occurred: {e}")

Start creating backend links for Analytics. Total of found pages: 360



  0%|                                                                                          | 0/360 [00:01<?, ?it/s]


An error occurred: name 'np' is not defined


ValueError: Error occurred: name 'np' is not defined

In [32]:
# display(new_dataframe.shape)
display(len(soup_dict))
display(len(keyword_dict))

9572

9308

In [28]:
# new_dataframe['id'].value_counts()

TypeError: 'NoneType' object is not subscriptable

In [19]:
new_dataframe['id'].value_counts()

# new_dataframe[new_dataframe['id']=="3670882403"]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df_unique = new_dataframe.drop_duplicates(subset=['id'])
df_unique


In [ ]:

with open("webscraper/webscrap_data/database.pkl" , "wb") as f:
    pickle.dump(df_unique, f)

In [ ]:
columns = new_dataframe.columns

for col in columns:
    null = new_dataframe[col].isna().sum()
    print(f"{round(null / len(new_dataframe[col]) * 100,1)}%    {col}:")

In [ ]:
keywords_input = ['Data%20Analyst']
keywords = []


# with open(f"webscrap_data/{filename}", 'rb') as file:
#        scraper_df = pickle.load(file)

In [ ]:
# https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0

    # first_url = f"https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&distance=25&f_JT=F&f_E=2%2C3&f_PP=106967730&position=1&pageNum=0"

In [ ]:
first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"



In [ ]:

# soup_dict = {}
# links = {}
# for key in keywords:
#     # first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"
#     key_name = key.replace('%20', " ")
        
#     # response = requests.get(first_url)
#     # response.status_code # 200 status code means OK!
#     # soup = BeautifulSoup(response.content, "html.parser")
    
#     # number_of_results = soup.find('span', class_="results-context-header__job-count").text
#     # numb = int(number_of_results.replace(",", "").replace("+", ""))
#     # # print(numb) # number of job post found 
    
#     # backend_call_url_list = create_backend_links(first_url, numb, key_name)
#     # with open(f'{key_name}_backend_urls.pkl', 'wb') as file:
#     #     pickle.dump(backend_call_url_list, file)
    
#     # id_list = get_id_dict(backend_call_url_list)
    
#     with tqdm(total=len(id_list), desc="Starting") as pbar:
#         for id in id_list:
#             dynamic_text = f"Progressing id: {id}"
#             pbar.set_description(dynamic_text)
#             if id not in id_control:
#                 try:
#                     scraper_df, id_control, small_soup = get_all_job_information(key_name, scraper_df, id, id_control)
#                     wait_time = randint(1,3000)
#                     pbar.set_description(f"Sleep {wait_time} seconds")
#                     sleep(wait_time/1000)
                    
#                     soup_dict[id] = small_soup
#                 except:
#                     print(f"Error when scraping data from id: {id}, temp backups are created in temp_data.")
#                     scraper_df_json = scraper_df.to_json('dataframe.json', orient='split', date_format='iso', indent=4)
#                     with open(f'temp_data/back_up_df.json', 'wb') as file:
#                         json.dump(scraper_df_json, file=file)
#                     with open('temp_data/small_soup_backup.json', 'w') as f:
#                         json.dump(small_soup, f)
#                     with open('temp_data/id_list_backup.pkl', "wb") as file:
#                         pickle.dump(links, file=file)
#             else:
#                 pbar.set_description(f"Will skip {id} because is already in the dataset.")
#             pbar.update(1)
    
#     current_date = datetime.now()
#     time = current_date.strftime("%Y-%m-%d_%H-%M-%S")
#     time

#     with open(f'webscrap_data/webscrap{str(time)}.pkl', 'wb') as file:
#         pickle.dump(scraper_df, file=file)
#     display(scraper_df.tail(5))
#     wait_time = randint(1,10000)
#     print("I will sleep for " + str(wait_time/1000) + " seconds.")
#     sleep(wait_time/1000)


In [ ]:
# use zlib.compress put into dict with id 
# pickle dict 

response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3717202675')


In [ ]:
soup_dict['3719620327']

In [ ]:
response = soup_dict['3719620327']

soup = BeautifulSoup(response.content, "html.parser")
soup

In [ ]:
soup.find("section",{"class":"show-more-less-html"}).text.strip()

In [36]:
soup_dict

{'3719620327': <Response [200]>,
 '3720365801': <Response [200]>,
 '3672508925': <Response [200]>,
 '3695466781': <Response [200]>,
 '3585738180': <Response [200]>,
 '3585734564': <Response [200]>,
 '3704916810': <Response [200]>,
 '3585734595': <Response [200]>,
 '3698396795': <Response [200]>,
 '3687813018': <Response [200]>,
 '3714114129': <Response [200]>,
 '3708672280': <Response [200]>,
 '3707596225': <Response [200]>,
 '3706371923': <Response [200]>,
 '3694298306': <Response [200]>,
 '3641771902': <Response [200]>,
 '3701510210': <Response [200]>,
 '3582381366': <Response [200]>,
 '3674803883': <Response [200]>,
 '3725401052': <Response [200]>,
 '3713235511': <Response [200]>,
 '3720128757': <Response [200]>,
 '3694036752': <Response [200]>,
 '3712833846': <Response [200]>,
 '3725985159': <Response [200]>,
 '3713068792': <Response [200]>,
 '3690755622': <Response [200]>,
 '3710653394': <Response [200]>,
 '3688655136': <Response [200]>,
 '3697730282': <Response [200]>,
 '36941130

In [41]:
soup_dict.keys()

dict_keys(['3719620327', '3720365801', '3672508925', '3695466781', '3585738180', '3585734564', '3704916810', '3585734595', '3698396795', '3687813018', '3714114129', '3708672280', '3707596225', '3706371923', '3694298306', '3641771902', '3701510210', '3582381366', '3674803883', '3725401052', '3713235511', '3720128757', '3694036752', '3712833846', '3725985159', '3713068792', '3690755622', '3710653394', '3688655136', '3697730282', '3694113016', '3679310304', '3704998318', '3711964211', '3704824751', '3722320318', '3719931304', '3699420235', '3582374933', '3674539739', '3715330729', '3586236970', '3463183538', '3698393103', '3638551593', '3712061807', '3720129659', '3590312287', '3624508875', '3709617062', '3709613515', '3705398931', '3723216390', '3654246723', '3698385819', '3718212892', '3725625612', '3726120710', '3714476818', '3710721213', '3714499999', '3706373793', '3724502483', '3652093754', '3720171940', '3709609985', '3661015665', '3706334246', '3725049678', '3725424456', '37220646

In [30]:
new_soup_dict = {}

In [44]:
for key in soup_dict:
    temp = {}
    
    dict2 = {}
    dict2["scrap_date"] = date.today()
    dict2["response"] = soup_dict[key]
    new_soup_dict[key] = dict2

In [45]:
new_soup_dict

{'3685669741': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3719620327': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3720365801': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3672508925': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3695466781': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3585738180': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3585734564': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3704916810': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3585734595': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3698396795': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3687813018': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},

In [46]:
with open("webscraper/webscrap_data/soup_dict.pkl", "wb") as f:
    pickle.dump(new_soup_dict, f)

In [25]:
id_list = mybib.get_id_list(backend_call_url_list)

  2%|█▉                                                                               | 15/640 [00:37<22:34,  2.17s/it]

  3%|██▏                                                                              | 17/640 [00:41<21:30,  2.07s/it]

  3%|██▎                                                                              | 18/640 [00:45<26:11,  2.53s/it]

  3%|██▌                                                                              | 20/640 [00:49<24:04,  2.33s/it]

  3%|██▋                                                                              | 21/640 [00:51<23:12,  2.25s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 640/640 [14:42<00:00,  1.38s/it]


In [28]:
len(id_list)

993